In [1]:
!pip install speechrecognition
!pip install pyttsx3
!pip install pywhatkit
!pip install pyaudio

In [18]:
import speech_recognition as sr
import pyttsx3
import datetime
import pywhatkit
import requests

# Initialize the speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Speech speed

# Text-to-speech function
def talk(text):
    print("Ruby:", text)
    engine.say(text)
    engine.runAndWait()

# Voice recognition function
def listen(prompt=False):
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 150  

    try:
        with sr.Microphone() as source:
            if prompt:
                talk(prompt)

            print("Adjusting for ambient noise...")
            recognizer.adjust_for_ambient_noise(source, duration=0.5)

            print("Listening now...")
            audio = recognizer.listen(source, timeout=10, phrase_time_limit=8)
            print("Processing your voice...")

            command = recognizer.recognize_google(audio)
            print("You said:", command)
            return command.lower()

    except sr.WaitTimeoutError:
        talk("You didn’t say anything. Please try again.")
    except sr.UnknownValueError:
        talk("Sorry, I couldn’t understand you.")
    except sr.RequestError:
        talk("I'm having trouble connecting to the speech service.")

    return ""

# Weather function using OpenWeatherMap API
def get_weather(city_name, api_key):
    print(f"City received: {city_name}")  # Debug print

    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city_name,
        "appid": api_key,
        "units": "metric"
    }

    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        
        if data["cod"] == 200:
            temp = data["main"]["temp"]
            desc = data["weather"][0]["description"]
            humidity = data["main"]["humidity"]
            report = f"The temperature in {city_name} is {temp}°C with {desc}. Humidity is {humidity}%."
            talk(report)
        else:
            talk("Sorry, I couldn't find the weather for that location.")

    except Exception as e:
        talk("Something went wrong while getting the weather.")
        print("Error:", str(e))

# Main function
def run_ruby():
    API_KEY = "20c670e06c07c66e95feb2d1d9d97412"  # Replace with your valid API key

    command = listen()

    if "hello" in command:
        talk("Hello there! I'm Ruby. How can I help you?")

    elif "your name" in command:
        talk("My name is Ruby. Nice to meet you!")

    elif "time" in command:
        current_time = datetime.datetime.now().strftime('%I:%M %p')
        talk(f"The current time is {current_time}")

    elif "date" in command:
        current_date = datetime.datetime.now().strftime('%B %d, %Y')
        talk(f"Today's date is {current_date}")

    elif "search" in command:
        topic = command.replace("search", "").strip()
        if topic:
            talk(f"Searching for {topic}")
            pywhatkit.search(topic)
        else:
            talk("Please tell me what to search for.")

    elif "weather" in command:
        city = listen(prompt="Please tell me the city name.")
        if not city:
            city = listen(prompt="Sorry, can you repeat the city name?")
        if city:
            get_weather(city, API_KEY)
        else:
            talk("I couldn't get the city name.")

    elif command != "":
        talk("Sorry, I didn't understand that. Can you repeat it?")
        
run_ruby()

Adjusting for ambient noise...
Listening now...
Processing your voice...
Ruby: Sorry, I couldn’t understand you.
